In [ ]:
pip install pandas spacy


In [ ]:
!python -m spacy download en_core_web_sm


2023-12-06 15:47:53.022249: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-06 15:47:53.029672: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-06 15:47:53.029793: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-06 15:47:53.061869: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-06 15:47:57.003523: W tensorflow/compiler/

In [ ]:
pip install pandas requests beautifulsoup4


In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os

# Load the input Excel file
input_file = 'Input (1).xlsx'
df = pd.read_excel(input_file)

# Function to extract article text from URL
def extract_article_text(url):
    try:
        # Send a GET request to the URL
        response = requests.get(url)
        response.raise_for_status()  # Raise an HTTPError for bad requests

        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')

        # Extract article title and text
        title = soup.title.text.strip()
        article_text = ' '.join([p.text.strip() for p in soup.find_all('p')])

        return title, article_text

    except Exception as e:
        print(f"Error extracting data from {url}: {str(e)}")
        return None, None

# Create a directory to store text files
output_directory = 'extracted_articles'
os.makedirs(output_directory, exist_ok=True)

# Iterate through each row in the DataFrame
for index, row in df.iterrows():
    url_id = row['URL_ID']
    url = row['URL']

    # Extract article text
    title, article_text = extract_article_text(url)

    # Save the extracted data to a text file
    if title and article_text:
        file_name = os.path.join(output_directory, f"{url_id}.txt")
        with open(file_name, 'w', encoding='utf-8') as file:
            file.write(f"Title: {title}\n\n")
            file.write(article_text)

print("Extraction and saving completed.")


Extraction and saving completed.


In [ ]:
!pip install textatistic


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for textatistic: filename=textatistic-0.0.1-py3-none-any.whl size=29047 sha256=f58bdd656b6d2d6191566f48242cd0805b3399d160cde97ff8da67b517a4e6b3
  Stored in directory: /root/.cache/pip/wheels/6e/31/c3/df8af4aa9ec3f671b0e8bc3bfa642c75918f2479ca4050f5f4
  Created wheel for pyhyphen: filename=PyHyphen-4.0.3-cp37-abi3-linux_x86_64.whl size=64708 sha256=4ee899277a86218ea9311613bce715467f3b06d8e09322984250612d5715c0b7
  Stored in directory: /root/.cache/pip/wheels/5e/8e/2c/099292ddfe67d4168785f44109ee79131c652ee5bdd66502b8
Successfully built textatistic pyhyphen


In [ ]:
!pip install python-docx


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 2.9 MB/s eta 0:00:00


In [ ]:
import os
import pandas as pd

# Directory containing text files
text_files_directory = '/content/extracted_articles'

# Output Excel file
output_excel_file = 'output.xlsx'

# Function to extract article title and text from a text file
def extract_article_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

        # Extracting the title (assuming it's on the first line)
        title = lines[0].replace('Title: ', '').strip()

        # Extracting the article text (assuming it's the rest of the lines)
        article_text = ' '.join(lines[1:]).strip()

    return title, article_text

# Iterate through each text file in the directory
data = {'File Name': [], 'Article Title': [], 'Article Text': []}

for filename in os.listdir(text_files_directory):
    if filename.endswith(".txt"):
        file_path = os.path.join(text_files_directory, filename)

        # Extract article data from the text file
        title, article_text = extract_article_data(file_path)

        # Add data to the dictionary
        data['File Name'].append(filename)
        data['Article Title'].append(title)
        data['Article Text'].append(article_text)

# Create a DataFrame from the dictionary
df = pd.DataFrame(data)

# Save the DataFrame to an Excel file
df.to_excel(output_excel_file, index=False, engine='openpyxl')

print(f"Conversion completed. Data saved to {output_excel_file}.")


Conversion completed. Data saved to output.xlsx.


In [ ]:
!pip install --upgrade textatistic


In [ ]:
!pip install textblob nltk
!python -m textblob.download_corpora


[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Unzipping corpora/conll2000.zip.
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.
Finished.


In [ ]:
import pandas as pd
import spacy
from textblob import TextBlob
from nltk import sent_tokenize, word_tokenize
from nltk.corpus import words
import os

# Load spaCy model
nlp = spacy.load("en_core_web_sm")

# Create a directory to store output files
output_directory = 'output_data'
os.makedirs(output_directory, exist_ok=True)

# Function to perform sentimental analysis and compute variables
def perform_text_analysis(article_text):
    # Sentiment analysis using TextBlob
    sentiment = TextBlob(article_text)
    positive_score = sentiment.sentiment.polarity
    negative_score = sentiment.sentiment.polarity * -1

    # Additional linguistic features using spaCy and NLTK
    doc = nlp(article_text)
    sentences = sent_tokenize(article_text)
    words_list = word_tokenize(article_text)

    avg_sentence_length = len(words_list) / len(sentences)
    percentage_of_complex_words = sum(1 for word in words_list if len(word) > 6) / len(words_list) * 100
    fog_index = 0.4 * (avg_sentence_length + percentage_of_complex_words)
    avg_number_of_words_per_sentence = len(words_list) / len(sentences)
    complex_word_count = sum(1 for word in words_list if len(word) > 6)
    word_count = len(words_list)
    syllable_per_word = sum(len(list(filter(str.isalpha, word))) for word in words_list) / len(words_list)
    personal_pronouns = sum(1 for token in doc if token.pos_ == 'PRON' and token.dep_ == 'nsubj')
    avg_word_length = sum(len(word) for word in words_list) / len(words_list)

    return [positive_score, negative_score, sentiment.sentiment.polarity,
            sentiment.sentiment.subjectivity, avg_sentence_length, percentage_of_complex_words,
            fog_index, avg_number_of_words_per_sentence, complex_word_count,
            word_count, syllable_per_word, personal_pronouns, avg_word_length]

# Load the extracted articles and perform analysis
input_directory = 'extracted_articles'
output_structure_file = 'Output Data Structure (2).xlsx'
output_df = pd.read_excel(output_structure_file)

for index, row in output_df.iterrows():
    url_id = row['URL_ID']
    input_file_path = os.path.join(input_directory, f"{url_id}.txt")

    # Read the extracted article text
    with open(input_file_path, 'r', encoding='utf-8') as file:
        article_text = file.read()

    # Perform sentimental analysis and compute variables
    analysis_results = perform_text_analysis(article_text)

    # Update the output DataFrame with computed variables
    output_df.iloc[index, 2:] = analysis_results

# Save the results to a new Excel file
output_file_path = os.path.join(output_directory, 'output_results_with_sentiment_analysis.xlsx')
output_df.to_excel(output_file_path, index=False)

print("Sentiment analysis and variable computation completed. Results saved in output_results_with_sentiment_analysis.xlsx.")


Sentiment analysis and variable computation completed. Results saved in output_results_with_sentiment_analysis.xlsx.
